In [ ]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
sp = spotipy.Spotify()

username = "zwamborn"
scope = "playlist-read-private"
client_id = "5711bc132b4c48ceb5bbd19cd65b1e63"
client_secret = "f507991961c948d8bf1b62ae6ef5ab15"
redirect_uri = "http://localhost"

def get_features(tracks):
    dataframe = []
    for i, item in enumerate(tracks['items']):
        try:
            track = item['track']
            #get audio features per track
            feat = (sp.audio_features(tracks=[track['id']])[0])
            #get track and artist info
            art_dict = {'artist':track['artists'][0]['name'], 'title':track['name']}
            #combine both dictionaries
            art_dict.update(feat)
            #make dataframe from dictionary
            dataframe.append(art_dict)
        except:
            print("Oh no!",sys.exc_info()[0],"happend")
    return dataframe

token = util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)

if token:
    top = 40
    sp = spotipy.Spotify(auth=token)
    playlists = sp.user_playlists(username)
    count = 0
    for playlist in playlists['items']:
        try:
            if playlist['owner']['id'] == username:
                #get tracks from all playlists
                results = sp.user_playlist(username, playlist['id'], fields="tracks,next")
                tracks = results['tracks']
                #make dataframe with track info and features
                data = get_features(tracks)
                frame = pd.DataFrame(data)
                frame = frame.set_index('title')
                #export 
                filename = playlist['name']+".csv"
                frame.to_csv(filename)
                while tracks['next']:
                    tracks = sp.next(tracks)
                    get_features(tracks)
        except:
            print("oh no, you've got no tracks")
else:
    print("Can't get token for", username)